# ECoG signals in humans and mice

In [ ]:
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
import LFPy
from lfpykit.eegmegcalc import FourSphereVolumeConductor
from lfpykit.models import CurrentDipoleMoment, RecMEAElectrode
from brainsignals.plotting_convention import mark_subplots, simplify_axes
from brainsignals import neural_simulations as ns

np.random.seed(1234)

head_colors = ["#ffb380", "#74abff", "#b3b3b3", "#c87137"]
human_radii = np.array([89000., 90000., 95000., 100000.])  # (µm)
human_sigmas = [0.276, 1.65, 0.01, 0.465]  # (S/m)

mice_radii = human_radii / 15
mice_sigmas = human_sigmas
eps = 1e-2

r_elecs_human = np.array([[0.], [0.], [human_radii[0] - eps]]).T
r_elecs_mice =  np.array([[0.], [0.], [mice_radii[0] - eps]]).T

human_4s = FourSphereVolumeConductor(r_elecs_human, human_radii, human_sigmas)
mice_4s = FourSphereVolumeConductor(r_elecs_mice, mice_radii, mice_sigmas)

def dipole_potential(elec_locs, dipole_pos, p):
    # Potential in infinite homogeneous medium
    r_ = elec_locs.T - dipole_pos
    V_e = 1000 * 1. / (4 * np.pi * human_sigmas[0]) * (np.dot(r_, p.T)
                    / np.linalg.norm(r_, axis=1) ** 3)
    return V_e


In [ ]:

dt = 2**-4
tstop = 20

# Define synapse parameters
synapse_params = {
    'e' : 0.,                   # reversal potential
    'syntype' : 'Exp2Syn',       # synapse type
    'tau1' : 0.1,                 # synaptic time constant
    'tau2' : 1.,                 # synaptic time constant
    'weight' : 0.001,            # synaptic weight
    'record_current' : False,    # record synapse current
}

cell_params = {
        'passive': True,
        'passive_parameters': {"g_pas": 1 / 30000,
                               "e_pas": -70.},
        'nsegs_method': "lambda_f",
        'Ra': 150,
        'cm': 1.0,
        'lambda_f': 100,
        'dt': dt,
        'tstart': -1,
        'tstop': tstop,
        'v_init': -70,
        'pt3d': True,
        'custom_code': [os.path.join(ns.allen_folder, "remove_axon.hoc")]
    }


In [ ]:
# Cell morphologies downloaded from the Allen Brain Atlas
morph_names = { 
               "L5_human_2": "H16.06.010.01.03.05.02_599474744_m.swc", 
               "L5_mice_1": "Tlx3-Cre_PL56_Ai14-338859.04.01.01_647469819_m.swc",
              }

morph_urls = {
    "H16.06.010.01.03.05.02_599474744_m.swc": 'https://celltypes.brain-map.org/api/v2/well_known_file_download/599474746',
    "Tlx3-Cre_PL56_Ai14-338859.04.01.01_647469819_m.swc": 'https://celltypes.brain-map.org/api/v2/well_known_file_download/647469823',
    }


dt = 2**-4
tstop = 20
np.random.seed(1234)
# Define synapse parameters
synapse_params = {
    'e': 0.,                   # reversal potential
    'syntype': 'Exp2Syn',       # synapse type
    'tau1': 0.1,                 # synaptic time constant
    'tau2': 1.,                 # synaptic time constant
    'weight': 0.001,            # synaptic weight
    'record_current': False,    # record synapse current
}

cell_params = {
        'passive': True,
        'passive_parameters': {"g_pas": 1 / 30000,
                               "e_pas": -70.},
        'nsegs_method': "lambda_f",
        'Ra': 150,
        'cm': 1.0,
        'lambda_f': 100,
        'dt': dt,
        'tstart': -1,
        'tstop': tstop,
        'v_init': -70,
        'pt3d': True,
        'custom_code': [os.path.join(ns.allen_folder, "remove_axon.hoc")]
    }

ecog_dict = {}
morph_dict = {}
synidx_dict = {}
tvec = None

for m_idx, morph_name in enumerate(morph_names.keys()):

    if "mice" in morph_name:
        foursphere = mice_4s
        radius_ = mice_radii[0]
    elif "human" in morph_name:
        foursphere = human_4s
        radius_ = human_radii[0]
    else:
        raise RuntimeError("Species not recognized!")
    
    ecog_dict[morph_name] = {}

    morph_file = morph_names[morph_name]
    morph_path = os.path.join(ns.allen_folder, morph_file)
    
    if not os.path.isfile(morph_path):
        print(f"Downloading morphology {morph_file}")
        from urllib.request import urlopen
        import ssl
        u = urlopen(morph_urls[morph_file], context=ssl._create_unverified_context())
        localFile = open(morph_path, 'wb')
        localFile.write(u.read())
        localFile.close()
    
    cell_params["morphology"] = morph_path

    cell = LFPy.Cell(**cell_params)
    ns.align_cell_to_axes(cell)

    synidxs = [cell.get_closest_idx(z=np.max(cell.z)),
               cell.get_closest_idx(z=np.max(cell.z)/2),
               0,
              ]
    
    cell.set_pos(z=radius_ - np.max(cell.z) - 100)
    morph_dict[morph_name] = [cell.x.copy(), cell.z.copy()]
    synidx_dict[morph_name] = synidxs
    del cell
    
    for s_idx, synidx in enumerate(synidxs):
        ecog_dict[morph_name][s_idx] = {}

        cell = LFPy.Cell(**cell_params)
        ns.align_cell_to_axes(cell)

        synapse_params["idx"] = synidx
        synapse = LFPy.Synapse(cell, **synapse_params)
        synapse.set_spike_times(np.array([1.]))

        cell.simulate(rec_imem=True, rec_vmem=True)
        print(morph_name, synidx, np.max(np.abs(cell.vmem[:, :] - cell.vmem[:, 0, None])))
        cdm = CurrentDipoleMoment(cell).get_transformation_matrix() @ cell.imem

        cell.set_pos(z=radius_ - np.max(cell.z) - 100)

        elec_params = {
        'sigma_G': human_sigmas[0],      # brain conductivity
        'sigma_S': human_sigmas[1],      # CSF cover conductivity
        'sigma_T': human_sigmas[0],      # brain conductivity
        'x': np.array([0]),  # electrode requires 1d vector of positions
        'y': np.array([0]),
        'z': np.array([radius_]),
        'h': radius_, 
        'method': "pointsource",
        }

        elec = RecMEAElectrode(cell=cell, **elec_params)
        ecog_MoI_s = elec.get_transformation_matrix() @ cell.imem * 1000

        elec_params['sigma_S'] = human_sigmas[0]
        elec = RecMEAElectrode(cell=cell, **elec_params)
        ecog_MoI_ih = elec.get_transformation_matrix() @ cell.imem * 1000

        multi_dipoles, dipole_locs = cell.get_multi_current_dipole_moments()

        ecog_mp = 0
        for dp_idx in range(len(multi_dipoles)):
            M_ = foursphere.get_transformation_matrix(dipole_locs[dp_idx])
            ecog_mp += 1000 * M_ @ multi_dipoles[dp_idx][:, :]# (uV)

        ecog_dict[morph_name][s_idx]["mp"] = ecog_mp
        ecog_dict[morph_name][s_idx]["MoI_s"] = ecog_MoI_s
        ecog_dict[morph_name][s_idx]["MoI_ih"] = ecog_MoI_ih
        
        tvec = cell.tvec.copy()
        del cell
        del elec
        del synapse

In [ ]:

plt.close("all")
fig = plt.figure(figsize=[6, 3.6])


for m_idx, morph_name in enumerate(morph_names.keys()):
    if "mice" in morph_name:
        radius_brain = mice_radii[0]
        radius_csf = mice_radii[1]
        marker_list1 = ["D", "E", "F"]
    elif "human" in morph_name:
        radius_brain = human_radii[0]
        radius_csf = human_radii[1]
        marker_list1 = ["A", "B", "C"]
    else:
        raise RuntimeError("Species not recognized!")
    
    for s_idx, syn_idx in enumerate(synidx_dict[morph_name]):

        ax1 = fig.add_axes([0.02 + 0.55 * m_idx, 0.73 - s_idx * 0.32, 0.18, 0.25], 
                           aspect=1, xlim=[-2000, 2000], 
                           ylim=[radius_brain - 2500, radius_brain + 500], 
                              xticks=[], yticks=[], frameon=False,)

        ax2 = fig.add_axes([0.28 + 0.55 * m_idx, 0.75 - s_idx * 0.32, 0.16, 0.19], 
                           ylabel="nV")

        ax1.plot([-500, -500], [radius_brain - 100, radius_brain - 1100], c='k', lw=1)
        ax1.text(-550, radius_brain - 600, "1 mm", ha="right", va="center")
        
        ax2.set_xlabel("time (ms)", labelpad=-1)
        ax1.add_patch(plt.Circle((0, 0), radius=radius_csf,
                                       color=head_colors[1],
                                       fill=True, ec='k', lw=.1))        
        
        ax1.add_patch(plt.Circle((0, 0), radius=radius_brain,
                                       color=head_colors[0],
                                       fill=True, ec='k', lw=.1))
        ax1.plot(morph_dict[morph_name][0].T, morph_dict[morph_name][1].T, c='k', 
                 rasterized=True)
        ax1.plot(morph_dict[morph_name][0][syn_idx].mean(), 
                 morph_dict[morph_name][1][syn_idx].mean(), 'o', c='b', ms=3)

        l_gt, = ax2.plot(tvec, 1000 * ecog_dict[morph_name][s_idx]["mp"][0], c='k', lw=2)
        l_s, = ax2.plot(tvec, 1000 * ecog_dict[morph_name][s_idx]["MoI_s"][0], c='gray', lw=1.)
        l_ih, = ax2.plot(tvec, 1000 * ecog_dict[morph_name][s_idx]["MoI_ih"][0], c='r', lw=1.)
        
        mark_subplots(ax1, marker_list1[s_idx], xpos=-0.05, ypos=1.05)
        
fig.legend([l_gt, l_s, l_ih], 
           ["four-sphere", "MoI with CSF", "homogeneous"], 
           ncol=3, frameon=False, loc=(0.2, -0.01))
simplify_axes(fig.axes)
plt.savefig("MoI_vs_4s.pdf")
